In [2]:
import pandas as pd
import pickle
import numpy as np

In [33]:
cpu_pass_LM = pd.read_csv('cpu_pass_LM.csv')
cpu_pass_LM

,name,price,core_count,core_clock,boost_clock,tdp,graphics,smt,codename,socket,lithography,cache,supported_mem,supported_chipset,release_date,pmprice,cpuMark,threadMark,category
0,AMD Ryzen 5 5600X,168.99,6,3.7,4.6,65,NaN,True,Vermeer,Socket AM4,7 nm,32 MB,DDR4,"AMD 300 Series*, AMD 400 Series, AMD 500 Series","Nov 5th, 2020",216.99,22088.0,3377.0,Desktop
1,AMD Ryzen 7 5800X,228.72,8,3.8,4.7,105,NaN,True,Vermeer,Socket AM4,7 nm,32 MB,DDR4,"AMD 300 Series*, AMD 400 Series, AMD 500 Series","Nov 5th, 2020",333.99,28368.0,3485.0,Desktop
2,Intel Core i7-13700K,381.99,16,3.4,5.4,125,Intel UHD Graphics 770,True,Raptor Lake-S,Socket 1700,10 nm,30 MB,"DDR4, DDR5","H610, B660, H670, Q670, Z690, W680, B760, H770...","Sep 27th, 2022",NaN,NaN,NaN,NaN
3,Intel Core i9-13900K,568.99,24,3.0,5.8,125,Intel UHD Graphics 770,True,Raptor Lake-S,Socket 1700,10 nm,36 MB,"DDR4, DDR5","H610, B660, H670, Q670, Z690, W680, B760, H770...","Sep 27th, 2022",NaN,NaN,NaN,NaN
4,AMD Ryzen 7 7800X3D,439.00,8,4.2,5.0,120,Radeon,True,Raphael,Socket AM5,5 nm,96 MB,DDR5,"X670E, X670, B650E, B650, A620","Jan 4th, 2023",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,AMD FX-4200,NaN,4,3.3,3.3,125,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890,AMD FX-6130,NaN,6,3.6,3.9,125,NaN,False,Zambezi,Socket AM3+,32 nm,8 MB,DDR3,NaN,"Oct 23rd, 2012",NaN,NaN,NaN,NaN
891,AMD Pro A6-7400B,NaN,2,3.5,3.9,65,Radeon R5 (on die),False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892,AMD Athlon X2 350,NaN,2,3.5,3.9,65,NaN,False,Brisbane,Socket AM2,65 nm,NaN,unknown,NaN,"Oct 8th, 2007",NaN,NaN,NaN,NaN


In [34]:
cpu_pass_LM['cache'] = cpu_pass_LM['cache'].str.replace(' MB', '')
cpu_pass_LM['lithography'] = cpu_pass_LM['lithography'].str.replace(' nm', '')

In [35]:
extras_cpu_pass = cpu_pass_LM.dropna(subset=['cache'])

In [36]:
with open('PDpassmark.pkl', 'rb') as file:
    model = pickle.load(file)

input_columns = ['core_count', 'boost_clock', 'tdp', 'smt', 'lithography']

df_new = extras_cpu_pass
for index, row in df_new.iterrows():
    if np.isnan(row['cpuMark']):
        X_pred = row[input_columns].values.reshape(1, -1)
        y_pred = model.predict(X_pred)      # making the prediction
        df_new.at[index, 'cpuMark'] = int(y_pred)
        df_new.at[index, 'passmarkPred'] = 'TRUE'
    else:
        df_new.at[index, 'passmarkPred'] = 'FALSE'

C:\Users\ZyroAki\AppData\Local\Temp\ipykernel_52772\2035959269.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.at[index, 'passmarkPred'] = 'FALSE'
C:\Users\ZyroAki\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\ZyroAki\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\ZyroAki\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitt

In [37]:
df_new

,name,price,core_count,core_clock,boost_clock,tdp,graphics,smt,codename,socket,lithography,cache,supported_mem,supported_chipset,release_date,pmprice,cpuMark,threadMark,category,passmarkPred
0,AMD Ryzen 5 5600X,168.99,6,3.7,4.6,65,NaN,True,Vermeer,Socket AM4,7,32,DDR4,"AMD 300 Series*, AMD 400 Series, AMD 500 Series","Nov 5th, 2020",216.99,22088.0,3377.0,Desktop,FALSE
1,AMD Ryzen 7 5800X,228.72,8,3.8,4.7,105,NaN,True,Vermeer,Socket AM4,7,32,DDR4,"AMD 300 Series*, AMD 400 Series, AMD 500 Series","Nov 5th, 2020",333.99,28368.0,3485.0,Desktop,FALSE
2,Intel Core i7-13700K,381.99,16,3.4,5.4,125,Intel UHD Graphics 770,True,Raptor Lake-S,Socket 1700,10,30,"DDR4, DDR5","H610, B660, H670, Q670, Z690, W680, B760, H770...","Sep 27th, 2022",NaN,42307.0,NaN,NaN,TRUE
3,Intel Core i9-13900K,568.99,24,3.0,5.8,125,Intel UHD Graphics 770,True,Raptor Lake-S,Socket 1700,10,36,"DDR4, DDR5","H610, B660, H670, Q670, Z690, W680, B760, H770...","Sep 27th, 2022",NaN,43348.0,NaN,NaN,TRUE
4,AMD Ryzen 7 7800X3D,439.00,8,4.2,5.0,120,Radeon,True,Raphael,Socket AM5,5,96,DDR5,"X670E, X670, B650E, B650, A620","Jan 4th, 2023",NaN,26121.0,NaN,NaN,TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,AMD Phenom II X4 905e,NaN,4,2.5,2.5,65,NaN,False,Deneb,Socket AM3,45,6,DDR3,"AMD 700 Series, AMD 800 Series, AMD 900 Series...","Jun 2nd, 2009",53.99,1895.0,1029.0,Desktop,FALSE
880,Intel Core i5-655K,NaN,2,3.2,3.2,73,Intel HD Graphics,True,Clarkdale,Socket 1156,32,4,DDR3,"H55, H57, P55","May 30th, 2010",77.99,2017.0,1446.0,Desktop,FALSE
883,AMD Phenom II X3 715 BE,NaN,3,2.8,2.8,95,NaN,False,Heka,Socket AM2+,45,6,"DDR2, DDR3","AMD 700 Series, AMD 800 Series, AMD 900 Series...","Oct 1st, 2010",NaN,968.0,NaN,NaN,TRUE
884,AMD Phenom II X4 970,NaN,4,3.5,3.5,125,NaN,False,Zosma,Socket AM3,45,6,"DDR2, DDR3","AMD 700 Series, AMD 800 Series, AMD 900 Series...","Oct 1st, 2010",59.88,2644.0,1412.0,Unknown,FALSE


In [38]:
df_new.to_csv('cpu_pass_impute.csv', index=False)